In [ ]:
import altair as alt
import pandas as pd
import pickle
import os
import numpy as np
import util
import gloop_net_analysis
from gloop_net_analysis import run_J, plot_kill_curves, plot_lag_distributions
from gloop_net_ltee import competition_assay
from igraph import Graph, EdgeSeq
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [ ]:
%load_ext autoreload
%autoreload 2

# Experiment (29/05/2022) - Effect of differing growth rate ($\mu$) on ltee

Simulating a LTEE experiment (In-Silico LTEE, or ISLTEE) when the phenotype is a lag distribution
 depends heavily on the growth rate.
In the limit of slow growth rate, the selection is for lag distributions with lower mean.
In the limit of fast growth, the selection is only on the height of the initial fraction of the lag distribution.
Here I use RCCN matrices (with constrained mean and variance) as genotypes which each give a lag distribution
 as the relevant phenotype
 In previous experiments (not ltee but selecting phenotype via an evolutionary algorithm with predefined fitness)
  selecting for the min (fast growth rate) had the odd (at first glance almost paradoxical) effect of lengthening
  considerably the lag tail (a persister phenotype?).
 We want to see if this reappears

## Results summary
The effect appears strongly for different mu. At $\mu = 1$ it's already nearly guaranteed (selection on the min gives the expected
higher min histogram + a long tail)


## Results location
experiments\LTEE\2022_05_29_T_w=100_diff_mu
experiments\LTEE\2022_05_29_T_w=2000_diff_mu






In [ ]:
def retrieve_mutant(mutant_dict, mutant_idx, ancestor_J):
    cur_idx = mutant_idx
    J = np.copy(ancestor_J)
    trace = [cur_idx]
    while cur_idx != 0:
        cur_idx = mutant_dict[cur_idx]["parent_id"]
        trace = [cur_idx] + trace

    for m in trace[1:]:
        mutated_node = mutant_dict[m]["mutated_node"]
        mutated_connections = mutant_dict[m]["mutated_connections"]
        row_col_mutation = 0 if mutant_dict[m]["row_col_mutation"]=="row" else 1
        if row_col_mutation:
            J[:, mutated_node] = mutated_connections
        else:
            J[mutated_node, :] = mutated_connections
    return J, len(trace)

In [ ]:
def plot_mutant_tree(mutant_dict, topology, final_N, title="Mutant Tree"):
    nr_vertices = len(mutant_dict) + 1
    nodes = [0] + list(mutant_dict.keys())
    edges = []


    for i in nodes[1:]:
        edges += [(mutant_dict[i]["parent_id"], i)]
    edges = edges[::-1]

    G = Graph(edges) # 2 stands for children number

    lay = G.layout_reingold_tilford(mode="in", root=0)

    position = {k: lay[k] for k in range(nr_vertices)}
    Y = [lay[k][1] for k in range(nr_vertices)]
    M = max(Y)

    # es = EdgeSeq(G) # sequence of edges
    E = [e.tuple for e in G.es] # list of edges

    L = len(position)
    Xn = [position[k][0] for k in range(L)]
    Yn = [2*M-position[k][1] for k in range(L)]
    Xe = []
    Ye = []
    for edge in E:
        Xe+=[position[edge[0]][0],position[edge[1]][0], None]
        Ye+=[2*M-position[edge[0]][1],2*M-position[edge[1]][1], None]


    # final_N_tot = sum(final_N[-1].values())
    get_final_conc = lambda i: int(final_N[-1][i]) if i in final_N[-1] else 0
    label_str = lambda i: f"{i}: (f_conc={get_final_conc(i):.0f}) Mutated {mutant_dict[i]['row_col_mutation']} #{mutant_dict[i]['mutated_node']} (size {topology[mutant_dict[i]['mutated_node']]})"
    descriptions = [f"Ancestor: (f_conc={get_final_conc(0):.0f})"] + [label_str(i) for i in nodes[1:]]

    mutated_cycle_size = [0] + [topology[mutant_dict[i]["mutated_node"]] for i in mutant_dict]
    final_conc = [get_final_conc(0)] + [get_final_conc(i) for i in mutant_dict]


    def make_annotations(pos, text, font_size=10, font_color='rgb(250,250,250)'):
        L=len(pos)
        if len(text)!=L:
            raise ValueError('The lists pos and text must have the same len')
        annotations = []
        for k in range(L):
            annotations.append(
                dict(
                    text=text[k], # or replace labels with a different list for the text within the circle
                    x=pos[k][0], y=2*M-position[k][1],
                    xref='x1', yref='y1',
                    font=dict(color=font_color, size=font_size),
                    showarrow=False)
            )
        return annotations


    fig = go.Figure()
    fig.add_trace(go.Scatter(x=Xe,
                       y=Ye,
                       mode='lines',
                       line=dict(color='rgb(210,210,210)', width=1),
                       hoverinfo='none'
                       ))
    fig.add_trace(go.Scatter(x=Xn,
                      y=Yn,
                      mode='markers',
                      name='bla',
                      marker=dict(symbol='circle-dot',
                                    # size=18,
                                    size=5*(np.log(np.array(mutated_cycle_size)+1)+1),
                                    # color=np.arange(nr_vertices),
                                    # color=mutated_cycle_size,
                                    color=final_conc,
                                    # color=[f"rgb({int(256*i/nr_vertices)},{int(256*i/nr_vertices)},{int(256*i/nr_vertices)})" for i in range(nr_vertices)],    #'#DB4551',
                                  # color_continuous_scale=["red", "green", "blue"],
                                    line=dict(color='rgb(50,50,50)', width=1)
                                    ),
                      text=descriptions,
                      hoverinfo='text',
                      opacity=0.8
                      ))

    axis = dict(showline=False, # hide axis line, grid, ticklabels and  title
                zeroline=False,
                showgrid=False,
                showticklabels=False,
                )

    fig.update_layout(title=title,
                  annotations=make_annotations(position, nodes),
                  font_size=12,
                  showlegend=False,
                  xaxis=axis,
                  yaxis=axis,
                  margin=dict(l=40, r=40, b=85, t=100),
                  hovermode='closest',
                  plot_bgcolor='rgb(248,248,248)'
                  )
    fig.show()

In [ ]:
topology, ancestor_J = np.load("topologies_and_Js\\topology_1.npy"), np.load("topologies_and_Js\\topology_1_J_1.npy")


In [ ]:
folder = r"C:\Users\Owner\PycharmProjects\GloopNetEvolution\experiments\LTEE\2022_05_29_T_w=100_diff_mu"

load_from_exp = lambda exp_name, file: pickle.load(
    open(fr"{folder}\{exp_name}\{file}", 'rb'))
save_to_exp = lambda exp_name, file, to_save: pickle.dump(to_save,
    open(fr"{folder}\{exp_name}\{file}", 'wb'))

def load_lag_and_mag(exp_name):
    lag_times = load_from_exp(exp_name, f"lag_times_mut_{mut_idx}")
    mag_records = load_from_exp(exp_name, f"mag_records_mut_{mut_idx}")
    return lag_times, mag_records

def gen_lag_and_mag(J, save_dir=""):
    lag_times = run_J(topology=topology, J=J, n_realizations_per_J=10000,
                      # T_ws=util.T_WS, record_mag=False)
                      T_ws=[100, 2000], record_mag=False)
                      # T_ws=[2000], record_mag=False)
    _, mag_records = run_J(topology=topology, J=J, n_realizations_per_J=100,
                      # T_ws=util.T_WS, record_mag=False)
                      T_ws=[100], record_mag=True)
    if save_dir:
        save_to_exp(save_dir, f"lag_times_mut_{mut_idx}", lag_times)
        save_to_exp(save_dir, f"mag_records_mut_{mut_idx}", mag_records)
    return lag_times, mag_records


for i, exp_name in enumerate(os.listdir(fr"{folder}")):
    # mu = float(exp_name.split("_")[-2].split("=")[1])
    final_N = load_from_exp(exp_name, "final_N")
    mutant_dict = load_from_exp(exp_name, "mutant_dict")
    # plot_mutant_tree(mutant_dict, topology, final_N)
    mut_idx = max(final_N[-1], key=lambda x: [final_N[-1][x]])
    mut_J, gen = (ancestor_J, 1) if mut_idx==0 else retrieve_mutant(mutant_dict, mut_idx, ancestor_J)
    lag_times, mag_records = gen_lag_and_mag(mut_J, save_dir=exp_name)
    # lag_times, mag_records = load_lag_and_mag(exp_name)


    # plot_kill_curves(lag_times, title=f"Kill curve - {exp_name}\n Mutant {mut_idx} (gen {gen})", save_dir=fr"figure_dump/kill_curve_{exp_name}_mut_{mut_idx}")
    # del lag_times[2000]
    # plot_lag_distributions(lag_times, x_max=150, y_max=300, title=f"Lag time distribution - {exp_name}\n Mutant {mut_idx} (gen({gen}))", save_dir=fr"figure_dump/lag_dist_{exp_name}_mut_{mut_idx}")
    plot_mag_trace(np.mean(mag_records[100], axis=-1), save_dir=fr"figure_dump/trace_{exp_name}_mut_{mut_idx}_",
                       lag_start_time =util.STABLE_T[1] + 100)




In [ ]:
def plot_mag_trace(mag_records, save_dir, lag_start_time):
    with plt.style.context("dark_background"):
        fig, ax = plt.subplots(1, 1, figsize=(20, 10))
    ax.set(ylim=(-0.2, 0.6), xlim=(lag_start_time, lag_start_time+800))
    ax.axhline(y=0, lw=1)
    for realization_i in range(len(mag_records)):
        trace = mag_records[realization_i]
        crossing_idx = np.argmax(trace[lag_start_time:]<=0) + lag_start_time
        p = ax.plot(np.arange(len(trace))[lag_start_time:crossing_idx+5], trace[lag_start_time:crossing_idx+5], lw=0.2)
        ax.axvline(x=crossing_idx, c=p[0].get_c(), ymax=0.25, lw=1)
    # ax.plot(np.arange(len(mag_records)), mag_records, lw=0.2)
    util.save_fig(save_dir, figure_dump=False, facecolor="k")


